# Information Retrieval Lab WiSe 2024/2025: Baseline Retrieval System

This Jupyter notebook serves as a baseline retrieval system that you can improve upon.
We use subsets of the MS MARCO datasets to retrieve passages of web documents.
We will show you how to create a software submission to TIRA from this notebook.

An overview of all corpora that we use in the current course is available at [https://tira.io/datasets?query=ir-lab-wise-2024](https://tira.io/datasets?query=ir-lab-wise-2024). The dataset IDs for loading the datasets are:

- `ir-lab-wise-2024/subsampled-ms-marco-deep-learning-20241201-training`: A subsample of the TREC 2019/2020 Deep Learning tracks on the MS MARCO v1 passage dataset. Use this dataset to tune your system(s).
- `ir-lab-wise-2024/subsampled-ms-marco-rag-20241202-training` (_work in progress_): A subsample of the TREC 2024 Retrieval-Augmented Generation track on the MS MARCO v2.1 passage dataset. Use this dataset to tune your system(s).
- `ir-lab-wise-2024/ms-marco-rag-20241203-test` (work in progress): The test corpus that we have created together in the course, based on the MS MARCO v2.1 passage dataset. We will use this dataset as the test dataset, i.e., evaluation scores become available only after the submission deadline.

### Step 1: Import libraries

We will use [tira](https://tira.io/), an information retrieval shared task platform, and [ir_dataset](https://ir-datasets.com/) for loading the datasets. Subsequently, we will build a retrieval system with [PyTerrier](https://github.com/terrier-org/pyterrier), an open-source search engine framework.

First, we need to install the required libraries.

In [ ]:
!pip3 install "tira>=0.0.139" ir-datasets "python-terrier==0.10.0"

Create an API client to interact with the TIRA platform (e.g., to load datasets and submit runs).

In [ ]:
from tira.third_party_integrations import ensure_pyterrier_is_loaded
from tira.rest_api_client import Client

ensure_pyterrier_is_loaded()
tira = Client()

### Step 2: Load the dataset

We load the dataset by its ir_datasets ID (as listed in the Readme). Just be sure to add the `irds:` prefix before the dataset ID to tell PyTerrier to load the data from ir_datasets.

In [3]:
import pyterrier as pt

pt_dataset = pt.get_dataset('irds:ir-lab-wise-2024/subsampled-ms-marco-deep-learning-20241201-training')

### Step 3: Build an index

We will then create an index from the documents in the dataset we just loaded.

In [ ]:
import os

indexer = pt.IterDictIndexer(
    index_path=os.getcwd() + os.sep + "index",
    meta={'docno': 50, 'text': 4096},
    # If an index already exists there, then overwrite it.
    overwrite=True,
)

index = indexer.index(pt_dataset.get_corpus_iter())

### Step 4: Define the retrieval pipeline

We will define a simple retrieval pipeline using just BM25 as a baseline. For details, refer to the PyTerrier [documentation](https://pyterrier.readthedocs.io) or [tutorial](https://github.com/terrier-org/ecir2021tutorial).

In [5]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25")

In [6]:
#as far as i can tell: SequentialDependence doesnt work for IterDictIndexer because you it doesnt have the "blocks" parameter which other indexers have
sdm = pt.rewrite.SequentialDependence()
sdm_pipeline = sdm >> bm25

In [7]:
bo1 = pt.rewrite.Bo1QueryExpansion(index)
bo1_pipeline = bm25 >> bo1 >> bm25

In [8]:
kl = pt.rewrite.KLQueryExpansion(index)
kl_pipeline = bm25 >> kl >> bm25

In [9]:
rm3 = pt.rewrite.RM3(index)
rm3_pipeline = bm25 >> rm3 >> bm25

### Step 5: Evaluate your run

In [11]:
pt.Experiment([bm25, bo1_pipeline, kl_pipeline, rm3_pipeline],
    pt_dataset.get_topics('text'),
    pt_dataset.get_qrels(),
    eval_metrics = ["map", "recip_rank", "ndcg_cut_10", "P_1", "P_5", "P_10"],
    names= ["BM25", "Bo1QueryExpansion", "KLQueryExpansion", "RM3"]
)

,name,map,recip_rank,ndcg_cut_10,P_1,P_5,P_10
0,BM25,0.412718,0.786653,0.489469,0.701031,0.622680,0.574227
1,Bo1QueryExpansion,0.458137,0.786259,0.509790,0.721649,0.645361,0.609278
2,KLQueryExpansion,0.456365,0.781917,0.505640,0.711340,0.649485,0.607216
3,RM3,0.455765,0.788708,0.506626,0.721649,0.643299,0.605155
